In [10]:
# blok 1 — Setup & konfigurasi (FAST preset untuk RTX 3050 Windows)

import os, math, random, time, warnings, gc, copy, atexit
from pathlib import Path
import numpy as np, pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, ConcatDataset
from torch.cuda.amp import GradScaler
import timm
from torchvision import transforms, datasets
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tqdm import tqdm

# ——— Anti-warning
os.environ["PYTHONWARNINGS"] = "ignore"
warnings.filterwarnings("ignore")
try:
    import multiprocessing as mp
    mp.set_start_method("spawn", force=True)
except RuntimeError:
    pass

# Paths
DATA_ROOT = Path("C:/Python/Template/Data Science/test/LOGIKA_UI")
SPLIT2_TRAIN = DATA_ROOT / "split2/train"
SPLIT2_VAL   = DATA_ROOT / "split2/val"
SPLIT2_TEST  = DATA_ROOT / "split2/test"
TEST_ORI_DIR = DATA_ROOT / "dsc-logika-ui-2025/Test/Test"
SAMPLE_SUB   = DATA_ROOT / "dsc-logika-ui-2025/sample_submission.csv"

# Output
OUT_DIR       = Path("C:/Python/Template/Data Science/test/LOGIKA_UI/output")
CKPT_DIR      = OUT_DIR / "checkpoints"
RETRAIN_DIR   = OUT_DIR / "retrained_full"
FIG_DIR       = OUT_DIR / "figs"
for d in [OUT_DIR, CKPT_DIR, RETRAIN_DIR, FIG_DIR]: d.mkdir(parents=True, exist_ok=True)

# —— Hyperparams & toggles (FAST preset aman untuk 4GB VRAM)
SEED = 1337
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMG_SIZE_TRAIN1 = 320     # freeze
IMG_SIZE_TRAIN2 = 384     # FT (boleh naik ke 416 kalau stabil)
BATCH_SIZE      = 16      # coba 16 dulu; kalau stabil, bisa 20
SPEED_MODE      = True

# ↑ Naikkan workers agar load gambar/augment lebih cepat (Windows aman di 2–4)
NUM_WORKERS     = 2
PREFETCH        = 2

FREEZE_LIGHT_AUG = True
ORIENT_AUG       = True

EPOCHS        = 10
FREEZE_EPOCHS = 1
LABEL_SMOOTH  = 0.05

# EMA dipisah per fase (hemat VRAM saat FT)
USE_EMA_FREEZE = True
USE_EMA_FT     = False
EMA_DECAY      = 0.999

EVAL_USE = "best"
BEST_BY  = "f1"

# OneCycle + LLRD
MAX_LR_HEAD   = 8e-4
MAX_LR_STAGE4 = 2e-4
MAX_LR_STAGE3 = 1e-4
MAX_LR_STAGE2 = 6e-5
MAX_LR_STAGE1 = 4e-5
WD         = 0.06
PCT_START  = 0.2
DIV_FACTOR = 20
FINAL_DIV  = 1000

# Opsional performa
CHANNELS_LAST = True
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision("high")

# Aktifkan TF32 (Ampere) untuk speed-up matmul/conv
if DEVICE.type == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

# Toggle gradient checkpointing (OFF untuk speed; ON jika butuh hemat VRAM)
USE_CHECKPOINT = False  # set True jika OOM di FT

# Seed & perf
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# Info GPU
print(f"Device: {DEVICE}")
if DEVICE.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))


Device: cuda
GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
# blok 2 — GeM, EMA, util (BN freeze, akurasi, F1, plot, helper LR)


class GeM(nn.Module):
    def __init__(self, p=3.0, eps=1e-6, learn_p=True):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1)*p) if learn_p else torch.tensor([p])
        self.eps = eps
    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        x = F.avg_pool2d(x, (x.size(-2), x.size(-1)))
        return x.pow(1./self.p)
    def __repr__(self):
        p = self.p.item() if isinstance(self.p, nn.Parameter) else float(self.p)
        return f"GeM(p={p:.3f})"

class ModelEMA:
    """Exponential Moving Average of model weights."""
    def __init__(self, model, decay=EMA_DECAY):
        self.model = model
        self.ema = copy.deepcopy(model).to(next(model.parameters()).device)
        self.ema.eval()
        self.decay = decay
        for p in self.ema.parameters():
            p.requires_grad_(False)
    @torch.no_grad()
    def update(self):
        msd, esd = self.model.state_dict(), self.ema.state_dict()
        for k in esd.keys():
            if esd[k].dtype.is_floating_point:
                esd[k].mul_(self.decay).add_(msd[k].detach(), alpha=1.0 - self.decay)
            else:
                esd[k].copy_(msd[k])

def set_bn_eval(m):
    if isinstance(m, (nn.BatchNorm2d, nn.SyncBatchNorm, nn.BatchNorm1d)):
        m.eval()

def accuracy_top1(output, target):
    with torch.no_grad():
        pred = output.argmax(1)
        return (pred == target).float().mean().item()

def compute_f1_macro(logits_cpu, labels_cpu):
    preds = logits_cpu.argmax(1).numpy()
    y = labels_cpu.numpy()
    return f1_score(y, preds, average="macro")

def plot_confusion(cm, classes, save_path):
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.tight_layout()
    plt.savefig(save_path); plt.close()

def lr_dict(optimizer):
    # mengembalikan lr per-group (s1,s2,s3,s4,head) atau 1 group saat freeze
    lrs = [pg["lr"] for pg in optimizer.param_groups]
    return lrs


In [ ]:
# blok 3 — Dataloader (PATCH: workers toggle + augment freeze/FT + aman Windows)

from torchvision.transforms import InterpolationMode
1 = randomsizecrop
2 = jitter
3 = contrast
4 = rotate
5 = mixup
6 = adas
7 = sasas
jumlah augment = [1,2,3,5,6,7]
def build_transforms(size, train=True, light=False):
    if train:
        if light:
            # — ringan (freeze): cepat tapi tetap robust
            tfs = [
                transforms.RandomResizedCrop(size=size, scale=(0.8, 1.0), interpolation=InterpolationMode.BICUBIC),
                transforms.RandomHorizontalFlip(),
            ]
            if ORIENT_AUG:
                tfs.insert(1, transforms.RandomVerticalFlip(p=0.2))
            tfs += [
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
            ]
            return transforms.Compose(tfs)
        else:
            # — FT: jika SPEED_MODE True, pakai augment lebih ringan agar batch pertama tidak macet di Windows
            if SPEED_MODE:
                tfs = [
                    transforms.RandomResizedCrop(size=size, scale=(0.8, 1.0), interpolation=InterpolationMode.BICUBIC),
                    transforms.RandomHorizontalFlip(),
                ]
                if ORIENT_AUG:
                    tfs.insert(1, transforms.RandomVerticalFlip(p=0.2))
                tfs += [
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
                    transforms.RandomErasing(p=0.10, value='random'),
                ]
                return transforms.Compose(tfs)
            else:
                return transforms.Compose([
                    transforms.RandomResizedCrop(size=size, scale=(0.7, 1.0), interpolation=InterpolationMode.BICUBIC),
                    transforms.RandomHorizontalFlip(),
                    transforms.ColorJitter(0.2,0.2,0.2,0.05),
                    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
                    transforms.RandomErasing(p=0.25, value='random')
                ])
    else:
        return transforms.Compose([
            transforms.Resize(int(size*1.14), interpolation=InterpolationMode.BICUBIC),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
        ])

def make_loader(dataset, batch, shuffle):
    kw = dict(
        batch_size=batch,
        shuffle=shuffle,
        pin_memory=(DEVICE.type=="cuda"),
        drop_last=shuffle
    )
    if NUM_WORKERS > 0:
        # Matikan persistent_workers di Windows (hindari deadlock saat ganti loader)
        use_persist = (os.name != "nt")
        kw.update(num_workers=NUM_WORKERS, prefetch_factor=PREFETCH, persistent_workers=use_persist)
    return DataLoader(dataset, **kw)

def make_loaders(train_dir, val_dir, size1=IMG_SIZE_TRAIN1, size2=IMG_SIZE_TRAIN2):
    tf_train1 = build_transforms(size1, train=True,  light=FREEZE_LIGHT_AUG)   # ← ringan saat freeze
    tf_train2 = build_transforms(size2, train=True,  light=False)
    tf_val    = build_transforms(size2, train=False, light=False)

    ds_train1 = datasets.ImageFolder(str(train_dir), transform=tf_train1)
    ds_train2 = datasets.ImageFolder(str(train_dir), transform=tf_train2)
    ds_val    = datasets.ImageFolder(str(val_dir),   transform=tf_val)

    classes = ds_train1.classes
    assert classes == ds_val.classes, "Kelas train & val tidak konsisten!"

    # Val batch: jangan 2x; jaga agar amanVRAM di 4GB
    val_batch = max(8, BATCH_SIZE)

    dl_train1 = make_loader(ds_train1, BATCH_SIZE,  True)
    dl_train2 = make_loader(ds_train2, BATCH_SIZE,  True)
    dl_val    = make_loader(ds_val,   val_batch,    False)
    return dl_train1, dl_train2, dl_val, classes, ds_train1.class_to_idx


In [ ]:
# blok 4 — Model, freeze, dan LLRD (PUSAT PILIH BACKBONE DI SINI)

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F

# ====== PILIH BACKBONE DI SINI ======
# 1: ConvNeXt-Tiny | 2: ConvNeXt-Small | 3: EfficientNet-B0 | 4: ResNet50-D | 5: CoAt-Lite-Mini
MODEL_ID = 2  # default: ConvNeXt-Small

# Peta backbone yang didukung
MODEL_ZOO = {
    1: ("convnext_tiny",          "ConvNeXt-Tiny"),
    2: ("convnext_small",         "ConvNeXt-Small"),
    3: ("tf_efficientnet_b0_ns",  "EfficientNet-B0 (NoisyStudent)"),
    4: ("resnet50d",              "ResNet50-D"),
    5: ("coat_lite_mini",         "CoAt-Lite-Mini"),
}

def _resolve_backbone(model_id: int):
    if model_id not in MODEL_ZOO:
        print(f"[WARN] MODEL_ID {model_id} tidak dikenali. Pakai 2 (ConvNeXt-Small).")
        model_id = 2
    timm_name, pretty = MODEL_ZOO[model_id]
    return model_id, timm_name, pretty

class GeM(nn.Module):
    def __init__(self, p: float = 3.0, eps: float = 1e-6, learn_p: bool = True):
        super().__init__()
        if learn_p:
            self.p = nn.Parameter(torch.ones(1) * p)
        else:
            self.register_buffer("p", torch.tensor([p], dtype=torch.float32))
        self.eps = eps
    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        x = F.avg_pool2d(x, (x.size(-2), x.size(-1)))
        return x.pow(1.0 / self.p)
    def extra_repr(self) -> str:
        # aman meski self.p adalah Parameter atau Buffer
        p_val = float(self.p.detach().cpu().item())
        return f"p={p_val:.3f}"

def set_bn_eval(m):
    if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.SyncBatchNorm)):
        m.eval()

class ClsNet(nn.Module):
    """
    Head seragam untuk berbagai CNN timm:
      backbone (timm, num_classes=0, global_pool="") → GeM → BNneck → Linear(num_classes)
    """
    def __init__(self, num_classes: int, model_id: int = MODEL_ID, pretrained: bool = True):
        super().__init__()
        self.model_id, self.timm_name, self.backbone_pretty = _resolve_backbone(model_id)

        # NOTE: global_pool="" + num_classes=0 → keluaran feature map [B, C, H, W]
        self.backbone = timm.create_model(
            self.timm_name, pretrained=pretrained,
            features_only=False, num_classes=0, global_pool=""
        )

        # Tentukan dimensi fitur (C) untuk head
        feat_dim = getattr(self.backbone, "num_features", None)
        if feat_dim is None:
            fi = getattr(self.backbone, "feature_info", None)
            if fi is not None:
                try:
                    feat_dim = fi.channels()[-1] if hasattr(fi, "channels") else fi[-1]["num_chs"]
                except Exception:
                    pass
        if feat_dim is None:
            raise RuntimeError(f"Tidak bisa menentukan dimensi fitur untuk {self.timm_name}")

        self.pool   = GeM(p=3.0, learn_p=True)
        self.bnneck = nn.BatchNorm1d(feat_dim, eps=1e-5, momentum=0.1)
        self.bnneck.bias.requires_grad_(False)
        self.head   = nn.Linear(feat_dim, num_classes, bias=True)

    def forward(self, x):
        f = self.backbone(x)              # [B, C, H, W]
        f = self.pool(f).flatten(1)       # [B, C]
        f = self.bnneck(f)                # [B, C]
        logits = self.head(f)             # [B, num_classes]
        return logits

def freeze_backbone(model: ClsNet, freeze: bool = True):
    for p in model.backbone.parameters():
        p.requires_grad_(not freeze)

def _chunk_llrd(params_list, chunks=4):
    """Fallback LLRD generik: belah parameter backbone berdasar urutan menjadi 4 tahap."""
    n = len(params_list)
    if n == 0:
        return [[], [], [], []]
    b = max(1, n // chunks)
    s1 = params_list[:b]
    s2 = params_list[b:2*b]
    s3 = params_list[2*b:3*b]
    s4 = params_list[3*b:]
    return [s1, s2, s3, s4]

def param_groups_llrd(model: ClsNet):
    """
    LLRD fleksibel:
      - Jika backbone punya pola 'stages.x' (ConvNeXt), pakai mapping spesifik.
      - Jika tidak, fallback: bagi 4 bagian berdasar urutan parameter backbone.
    """
    stage1, stage2, stage3, stage4 = [], [], [], []
    seen = set()
    names = [n for n,_ in model.backbone.named_parameters()]

    if any("stages." in n for n in names):
        # Mapping spesifik ConvNeXt
        for n, p in model.backbone.named_parameters():
            if not p.requires_grad or id(p) in seen:
                continue
            if   "stages.3" in n: stage4.append(p)
            elif "stages.2" in n: stage3.append(p)
            elif "stages.1" in n: stage2.append(p)
            else:                  stage1.append(p)
            seen.add(id(p))
    else:
        # Fallback generik (EfficientNet/ResNet/CoAt-lite, dll.)
        linearized = []
        for n, p in model.backbone.named_parameters():
            if not p.requires_grad or id(p) in seen:
                continue
            linearized.append(p); seen.add(id(p))
        stage1, stage2, stage3, stage4 = _chunk_llrd(linearized, chunks=4)

    # head params (bnneck + head) — pastikan tidak overlap
    head_params = []
    for n, p in model.named_parameters():
        if ("bnneck" in n or "head" in n) and p.requires_grad and id(p) not in seen:
            head_params.append(p); seen.add(id(p))

    # Validasi unik
    uniq = set(id(p) for g in [stage1,stage2,stage3,stage4,head_params] for p in g)
    total = sum(len(g) for g in [stage1,stage2,stage3,stage4,head_params])
    assert len(uniq) == total, "Param duplicate terdeteksi di LLRD groups."

    groups = [
        {"params": stage1, "lr": MAX_LR_STAGE1, "weight_decay": WD},
        {"params": stage2, "lr": MAX_LR_STAGE2, "weight_decay": WD},
        {"params": stage3, "lr": MAX_LR_STAGE3, "weight_decay": WD},
        {"params": stage4, "lr": MAX_LR_STAGE4, "weight_decay": WD},
        {"params": head_params, "lr": MAX_LR_HEAD,  "weight_decay": WD},
    ]
    return groups


In [14]:
# # blok 5 — Training tunggal (pakai MODEL_ID untuk pilih backbone)

# import time, copy, contextlib
# from tqdm import tqdm
# from sklearn.metrics import f1_score
# from torch import amp  # AMP universal (cpu/cuda)

# # util bar & metric
# def _bar(desc): return dict(desc=desc, ncols=120, bar_format="{l_bar}{bar:50}{r_bar}")
# def _acc(logits, y):  return (logits.argmax(1) == y).float().mean().item()
# def _f1_macro(lg, lb): 
#     p = lg.argmax(1).numpy(); y = lb.numpy()
#     return f1_score(y, p, average="macro")
# def _lr_list(optim): return [pg["lr"] for pg in optim.param_groups]
# def _set_bn_eval(m):
#     if isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.SyncBatchNorm)): m.eval()

# class ModelEMA:
#     def __init__(self, m, decay=EMA_DECAY):
#         self.model = m
#         self.ema = copy.deepcopy(m).to(next(m.parameters()).device).eval()
#         self.decay = decay
#         for p in self.ema.parameters(): p.requires_grad_(False)
#     @torch.no_grad()
#     def update(self):
#         msd = self.model.state_dict()
#         for k, ev in self.ema.state_dict().items():
#             v = msd[k]
#             if ev.dtype.is_floating_point: ev.mul_(self.decay).add_(v.detach(), alpha=1.0-self.decay)
#             else: ev.copy_(v)

# # — data & model
# dl_train1, dl_train2, dl_val, CLASSES, _ = make_loaders(SPLIT2_TRAIN, SPLIT2_VAL)
# NUM_CLASSES = len(CLASSES)

# # === INSTANSIASI MODEL DENGAN PILIHAN BACKBONE ===
# model = ClsNet(num_classes=NUM_CLASSES, model_id=MODEL_ID, pretrained=True).to(DEVICE)
# print(f"[Model] Backbone: {model.backbone_pretty} ({model.timm_name}) | MODEL_ID={MODEL_ID}")

# if CHANNELS_LAST && DEVICE.type == "cuda":
#     model = model.to(memory_format=torch.channels_last)

# # (opsional) grad checkpointing mengikuti flag global
# if globals().get("USE_CHECKPOINT", False):
#     try:
#         model.backbone.set_grad_checkpointing(True)
#     except Exception:
#         pass

# criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)
# scaler    = GradScaler(enabled=(DEVICE.type == "cuda"))
# ema       = ModelEMA(model, EMA_DECAY) if USE_EMA_FREEZE else None

# print(f"Trainable params (before freeze): {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
# if DEVICE.type == "cuda":
#     assert next(model.parameters()).is_cuda, "Model tidak ada di CUDA!"
# else:
#     print("[INFO] CUDA tidak tersedia, training di CPU.")

# def _to_device(imgs):
#     t = imgs.to(DEVICE, non_blocking=True)
#     if CHANNELS_LAST and DEVICE.type == "cuda":
#         t = t.to(memory_format=torch.channels_last)
#     return t

# def train_one_epoch(loader, optimizer, scheduler=None, freeze_bn=False, desc="Train"):
#     model.train()
#     if freeze_bn: model.apply(_set_bn_eval)

#     loss_sum = acc_sum = n = 0.0
#     all_logits, all_labels = [], []
#     pbar = tqdm(loader, **_bar(desc))
#     for imgs, labels in pbar:
#         imgs, labels = _to_device(imgs), labels.to(DEVICE, non_blocking=True)

#         optimizer.zero_grad(set_to_none=True)
#         with amp.autocast(device_type=DEVICE.type):
#             logits = model(imgs)
#             loss = criterion(logits, labels)

#         if scaler.is_enabled():
#             scaler.scale(loss).backward()
#             scaler.step(optimizer); scaler.update()
#         else:
#             loss.backward(); optimizer.step()

#         if scheduler is not None: scheduler.step()
#         if ema is not None: ema.update()

#         bs = imgs.size(0)
#         loss_sum += loss.item()*bs
#         acc_sum  += _acc(logits, labels)*bs
#         n += bs
#         all_logits.append(logits.detach().float().cpu())
#         all_labels.append(labels.detach().cpu())

#         pbar.set_postfix(loss=f"{loss_sum/max(n,1):.4f}",
#                          acc=f"{acc_sum/max(n,1):.4f}",
#                          lr=f"{_lr_list(optimizer)[-1]:.2e}")
#     pbar.close()
#     f1m = _f1_macro(torch.cat(all_logits), torch.cat(all_labels))
#     return loss_sum/max(n,1), acc_sum/max(n,1), f1m

# @torch.no_grad()
# def validate(loader, freeze_bn=False, desc="Val"):
#     eval_model = ema.ema if (ema is not None) else model
#     eval_model.eval()
#     if freeze_bn: eval_model.apply(_set_bn_eval)

#     loss_sum = acc_sum = n = 0.0
#     all_logits, all_labels = [], []
#     pbar = tqdm(loader, **_bar(desc))
#     for imgs, labels in pbar:
#         imgs, labels = _to_device(imgs), labels.to(DEVICE, non_blocking=True)
#         with amp.autocast(device_type=DEVICE.type):
#             logits = eval_model(imgs); loss = criterion(logits, labels)
#         bs = imgs.size(0)
#         loss_sum += loss.item()*bs
#         acc_sum  += _acc(logits, labels)*bs
#         n += bs
#         all_logits.append(logits.float().cpu()); all_labels.append(labels.cpu())
#         pbar.set_postfix(loss=f"{loss_sum/max(n,1):.4f}", acc=f"{acc_sum/max(n,1):.4f}")
#     pbar.close()
#     f1m = _f1_macro(torch.cat(all_logits), torch.cat(all_labels))
#     return loss_sum/max(n,1), acc_sum/max(n,1), f1m

# # ==== Fase FREEZE ====
# freeze_backbone(model, True)
# opt = torch.optim.AdamW([{"params":[p for p in model.parameters() if p.requires_grad],
#                           "lr": MAX_LR_HEAD, "weight_decay": WD}])
# sched = torch.optim.lr_scheduler.OneCycleLR(
#     opt, max_lr=MAX_LR_HEAD, steps_per_epoch=len(dl_train1),
#     epochs=max(FREEZE_EPOCHS,1), pct_start=min(PCT_START,0.5),
#     div_factor=DIV_FACTOR, final_div_factor=FINAL_DIV
# )

# best_score = -1.0
# BEST_CKPT_PATH = CKPT_DIR / "best.pt"
# print("\n[Train] Fase Linear Probing (freeze backbone)")
# for ep in range(1, FREEZE_EPOCHS+1):
#     tl, ta, tf = train_one_epoch(dl_train1, opt, sched, freeze_bn=True, desc=f"Train(FREEZE) {ep}/{FREEZE_EPOCHS}")
#     vl, va, vf = validate(dl_val,       freeze_bn=True, desc=f"Val(FREEZE) {ep}/{FREEZE_EPOCHS}")
#     print(f"[Freeze {ep}/{FREEZE_EPOCHS}] loss={tl:.4f} acc={ta:.4f} f1={tf:.4f} | VAL loss={vl:.4f} acc={va:.4f} f1={vf:.4f}")
#     if vf > best_score:
#         best_score = vf
#         torch.save({"model": (ema.ema if ema is not None else model).state_dict(), "classes": CLASSES}, BEST_CKPT_PATH)
#         print(f"  -> Saved BEST (VAL F1 {best_score:.4f}) to {BEST_CKPT_PATH}")

# # ==== Fase FULL FINE-TUNE ====
# freeze_backbone(model, False)
# ema = ModelEMA(model, EMA_DECAY) if USE_EMA_FT else None
# opt = torch.optim.AdamW(param_groups_llrd(model))
# sched = torch.optim.lr_scheduler.OneCycleLR(
#     opt, max_lr=[pg["lr"] for pg in opt.param_groups], steps_per_epoch=len(dl_train2),
#     epochs=max(EPOCHS-FREEZE_EPOCHS,1), pct_start=PCT_START, div_factor=DIV_FACTOR, final_div_factor=FINAL_DIV
# )

# print("\n[Train] Fase Full Fine-Tune (unfreeze + LLRD)")
# for ep in range(1, (EPOCHS-FREEZE_EPOCHS)+1):
#     tl, ta, tf = train_one_epoch(dl_train2, opt, sched, freeze_bn=(BATCH_SIZE<16), desc=f"Train(FT) {ep}/{EPOCHS-FREEZE_EPOCHS}")
#     vl, va, vf = validate(dl_val,       freeze_bn=False, desc=f"Val(FT) {ep}/{EPOCHS-FREEZE_EPOCHS}")
#     lrs = _lr_list(opt)
#     print(f"[FT {ep}/{EPOCHS-FREEZE_EPOCHS}] loss={tl:.4f} acc={ta:.4f} f1={tf:.4f} | VAL loss={vl:.4f} acc={va:.4f} f1={vf:.4f} | "
#           f"LR s1={lrs[0]:.1e} s2={lrs[1]:.1e} s3={lrs[2]:.1e} s4={lrs[3]:.1e} head={lrs[4]:.1e}")

# LAST_CKPT_PATH = CKPT_DIR / "last.pt"
# torch.save({"model": model.state_dict(), "classes": CLASSES}, LAST_CKPT_PATH)
# print(f"Saved LAST to {LAST_CKPT_PATH}")


In [15]:
# # blok 6 — Evaluasi split2/test (mandiri, tanpa dependency evaluate())
# import time
# from tqdm import tqdm
# import torch
# import torch.nn as nn
# from sklearn.metrics import classification_report, confusion_matrix, f1_score
# import seaborn as sns, matplotlib.pyplot as plt

# # helper kecil agar bar konsisten & metrik beres
# def _bar(desc): return dict(desc=desc, ncols=120, bar_format="{l_bar}{bar:50}{r_bar}")

# def _acc(logits, y):
#     with torch.no_grad():
#         return (logits.argmax(1) == y).float().mean().item()

# def _f1_macro(logits_cpu, labels_cpu):
#     p = logits_cpu.argmax(1).numpy()
#     y = labels_cpu.numpy()
#     return f1_score(y, p, average="macro")

# def _set_bn_eval(m):
#     if isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.SyncBatchNorm)):
#         m.eval()

# @torch.no_grad()
# def _eval_loop(model, loader, criterion, freeze_bn=False, desc="Eval split2/test"):
#     model.eval()
#     if freeze_bn: model.apply(_set_bn_eval)

#     loss_sum = acc_sum = n = 0.0
#     all_logits, all_labels = [], []
#     pbar = tqdm(loader, **_bar(desc))
#     for imgs, labels in pbar:
#         imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
#         with autocast():
#             logits = model(imgs)
#             loss = criterion(logits, labels)
#         bs = imgs.size(0)
#         loss_sum += loss.item()*bs
#         acc_sum  += _acc(logits, labels)*bs
#         n += bs
#         all_logits.append(logits.float().cpu())
#         all_labels.append(labels.cpu())
#         pbar.set_postfix(loss=f"{loss_sum/max(n,1):.4f}",
#                          acc=f"{acc_sum/max(n,1):.4f}")
#     pbar.close()
#     logits_cpu = torch.cat(all_logits) if all_logits else torch.empty(0)
#     labels_cpu = torch.cat(all_labels) if all_labels else torch.empty(0, dtype=torch.long)
#     f1m = _f1_macro(logits_cpu, labels_cpu) if len(labels_cpu)>0 else 0.0
#     return loss_sum/max(n,1), acc_sum/max(n,1), f1m, logits_cpu, labels_cpu

# # ---- siapkan ckpt & loader test ----
# # pilih ckpt untuk evaluasi: best vs last (fallback ke best jika variabel belum ada)
# _ckpt_best = (CKPT_DIR / "best.pt") if "BEST_CKPT_PATH" not in globals() else BEST_CKPT_PATH
# _ckpt_last = (CKPT_DIR / "last.pt") if "LAST_CKPT_PATH" not in globals() else LAST_CKPT_PATH
# ckpt_path  = _ckpt_best if globals().get("EVAL_USE","best") == "last" else _ckpt_last
# print(f"[Eval] Using checkpoint: {ckpt_path}")

# # build model & load weights (pakai kelas yang sama seperti blok 5)
# model_eval = ClsNet(num_classes=len(CLASSES), pretrained=False).to(DEVICE)
# ck = torch.load(ckpt_path, map_location=DEVICE)
# model_eval.load_state_dict(ck["model"])

# # dataloader test (berlabel)
# tf_test = build_transforms(IMG_SIZE_TRAIN2, train=False)
# ds_test = datasets.ImageFolder(str(SPLIT2_TEST), transform=tf_test)
# assert ds_test.classes == CLASSES, "Kelas split2/test tidak cocok dengan model!"
# # NUM_WORKERS & PREFETCH mengikuti blok 1
# kwargs = dict(batch_size=BATCH_SIZE*2, shuffle=False, pin_memory=(DEVICE.type=="cuda"))
# if globals().get("NUM_WORKERS", 0) > 0:
#     kwargs.update(num_workers=NUM_WORKERS, prefetch_factor=PREFETCH, persistent_workers=True)
# dl_test = DataLoader(ds_test, **kwargs)

# # run evaluasi
# criterion = nn.CrossEntropyLoss()
# t0 = time.time()
# loss, acc, f1m, logits, labels = _eval_loop(model_eval, dl_test, criterion, freeze_bn=False)
# print(f"[SPLIT2/TEST] loss={loss:.4f} acc={acc:.4f} f1={f1m:.4f} | {time.time()-t0:.1f}s")

# # simpan classification report & confusion matrix
# y_pred = logits.argmax(1).numpy()
# y_true = labels.numpy()
# report = classification_report(y_true, y_pred, target_names=CLASSES, digits=4)
# print(report)

# (FIG_DIR).mkdir(parents=True, exist_ok=True)
# with open(FIG_DIR / "classification_report.txt", "w") as f:
#     f.write(report)

# cm = confusion_matrix(y_true, y_pred)
# plt.figure(figsize=(8,6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
#             xticklabels=CLASSES, yticklabels=CLASSES)
# plt.xlabel("Predicted"); plt.ylabel("True"); plt.tight_layout()
# plt.savefig(FIG_DIR / "confusion_matrix.png"); plt.close()
# print(f"[Eval] Confusion matrix saved to {FIG_DIR/'confusion_matrix.png'}")


In [16]:
# blok 7 — Retrain 100% (gabung split2 train+val, opsional test) → simpan retrained_full.pt
# - Aman Windows/RTX3050: non-persistent workers, AMP universal, channels_last
# - Tidak melatih dari TEST secara default (hindari data leakage)
# - EMA per-fase mengikuti USE_EMA_FREEZE / USE_EMA_FT dari blok 1 (fallback disediakan)
# - Mendukung pilihan backbone via MODEL_ID (lihat blok 4)

import os, gc, time, copy
import torch
from torch import amp
from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets
from tqdm import tqdm
from sklearn.metrics import f1_score

# ===== Util =====
def _bar(desc): return dict(desc=desc, ncols=120, bar_format="{l_bar}{bar:50}{r_bar}")
def _acc(logits, y): return (logits.argmax(1) == y).float().mean().item()
def _f1_macro(lg, lb):
    p = lg.argmax(1).numpy(); y = lb.numpy()
    return f1_score(y, p, average="macro")
def _set_bn_eval(m):
    if isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.SyncBatchNorm)): m.eval()
def _lr_list(optim): return [pg["lr"] for pg in optim.param_groups]

# ===== EMA lokal (by-key) =====
if "ModelEMA" not in globals():
    class ModelEMA:
        def __init__(self, m, decay=EMA_DECAY):
            self.model = m
            self.ema = copy.deepcopy(m).to(next(m.parameters()).device).eval()
            self.decay = decay
            for p in self.ema.parameters(): p.requires_grad_(False)
        @torch.no_grad__()
        def update(self):
            msd = self.model.state_dict()
            for k, ev in self.ema.state_dict().items():
                v = msd[k]
                if ev.dtype.is_floating_point: ev.mul_(self.decay).add_(v.detach(), alpha=1.0-self.decay)
                else: ev.copy_(v)

# ===== Konfigurasi aman =====
INCLUDE_TEST_IN_TRAIN = False  # ← default: JANGAN latih dari TEST (hindari leakage)
USE_EMA_FREEZE = globals().get("USE_EMA_FREEZE", True)
USE_EMA_FT     = globals().get("USE_EMA_FT", False)

# ===== Dataset gabungan (freeze & finetune beda transform) =====
tf_freeze = build_transforms(IMG_SIZE_TRAIN1, train=True,  light=globals().get("FREEZE_LIGHT_AUG", False))
tf_finet  = build_transforms(IMG_SIZE_TRAIN2, train=True,  light=False)

ds_tr1 = datasets.ImageFolder(str(SPLIT2_TRAIN), tf_freeze)
ds_vl1 = datasets.ImageFolder(str(SPLIT2_VAL),   tf_freeze)

ds_tr2 = datasets.ImageFolder(str(SPLIT2_TRAIN), tf_finet)
ds_vl2 = datasets.ImageFolder(str(SPLIT2_VAL),   tf_finet)

if INCLUDE_TEST_IN_TRAIN:
    # HANYA aktifkan jika folder TEST berstruktur per-kelas (identik train/val).
    ds_te1 = datasets.ImageFolder(str(SPLIT2_TEST), tf_freeze)
    ds_te2 = datasets.ImageFolder(str(SPLIT2_TEST), tf_finet)
else:
    ds_te1 = None
    ds_te2 = None

CLASSES_FULL = ds_tr1.classes
assert CLASSES_FULL == ds_vl1.classes, "Kelas train & val tidak konsisten!"
if INCLUDE_TEST_IN_TRAIN:
    assert CLASSES_FULL == ds_te1.classes, "Kelas TEST tidak konsisten/berbeda struktur!"

# Full data untuk training 100%
full_list1 = [ds_tr1, ds_vl1] + ([ds_te1] if (INCLUDE_TEST_IN_TRAIN and ds_te1 is not None) else [])
full_list2 = [ds_tr2, ds_vl2] + ([ds_te2] if (INCLUDE_TEST_IN_TRAIN and ds_te2 is not None) else [])

ds_full1 = ConcatDataset(full_list1)  # fase freeze (size kecil/augment ringan)
ds_full2 = ConcatDataset(full_list2)  # fase finetune (ukuran besar/augment penuh)

# ===== DataLoader (non-persistent di Windows) =====
def _mk_loader(ds, batch, shuffle):
    kw = dict(batch_size=batch,
              shuffle=shuffle,
              pin_memory=(DEVICE.type=="cuda"),
              drop_last=shuffle)
    if globals().get("NUM_WORKERS", 0) > 0:
        use_persist = (os.name != "nt")  # Windows → False (aman)
        kw.update(num_workers=NUM_WORKERS,
                  prefetch_factor=PREFETCH,
                  persistent_workers=use_persist)
    return DataLoader(ds, **kw)

dl_full1 = _mk_loader(ds_full1, BATCH_SIZE, True)
dl_full2 = _mk_loader(ds_full2, BATCH_SIZE, True)

# ===== Model/criterion/scaler =====
model_full = ClsNet(num_classes=len(CLASSES_FULL), model_id=MODEL_ID, pretrained=True).to(DEVICE)
print(f"[Retrain-100%] Backbone: {model_full.backbone_pretty} ({model_full.timm_name}) | MODEL_ID={MODEL_ID}")

if globals().get("CHANNELS_LAST", True) and DEVICE.type=="cuda":
    model_full = model_full.to(memory_format=torch.channels_last)

# (Opsional) Grad checkpointing (hemat VRAM; sedikit lebih lambat)
if globals().get("USE_CHECKPOINT", False):
    try:
        model_full.backbone.set_grad_checkpointing(True)
    except Exception:
        pass

criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)
scaler    = GradScaler(enabled=(DEVICE.type == "cuda"))

def _to_device(imgs):
    t = imgs.to(DEVICE, non_blocking=True)
    if globals().get("CHANNELS_LAST", True) and DEVICE.type=="cuda":
        t = t.to(memory_format=torch.channels_last)
    return t

# ===== Loop train (tanpa validasi; metrik dicetak di data train gabungan) =====
def _train_epoch(loader, optimizer, scheduler=None, freeze_bn=False, desc="FULL-Train", ema_obj=None):
    model_full.train()
    if freeze_bn: model_full.apply(_set_bn_eval)
    loss_sum = acc_sum = n = 0.0
    all_logits, all_labels = [], []
    pbar = tqdm(loader, **_bar(desc))
    for imgs, labels in pbar:
        imgs, labels = _to_device(imgs), labels.to(DEVICE, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with amp.autocast(device_type=DEVICE.type):
            logits = model_full(imgs)
            loss = criterion(logits, labels)
        if scaler.is_enabled():
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()
        if scheduler is not None: scheduler.step()
        if ema_obj is not None: ema_obj.update()

        bs = imgs.size(0)
        loss_sum += loss.item()*bs
        acc_sum  += _acc(logits, labels)*bs
        n += bs
        all_logits.append(logits.detach().float().cpu())
        all_labels.append(labels.detach().cpu())
        pbar.set_postfix(loss=f"{loss_sum/max(n,1):.4f}",
                         acc=f"{acc_sum/max(n,1):.4f}",
                         lr=f"{_lr_list(optimizer)[-1]:.2e}")
    pbar.close()
    f1m = _f1_macro(torch.cat(all_logits), torch.cat(all_labels))
    return loss_sum/max(n,1), acc_sum/max(n,1), f1m

# ====== Fase 1: FREEZE (Linear Probing di full data) ======
ema_full = (ModelEMA(model_full, EMA_DECAY) if USE_EMA_FREEZE else None)
freeze_backbone(model_full, True)
opt = torch.optim.AdamW([{"params":[p for p in model_full.parameters() if p.requires_grad],
                          "lr": MAX_LR_HEAD, "weight_decay": WD}])
sched = torch.optim.lr_scheduler.OneCycleLR(
    opt, max_lr=MAX_LR_HEAD, steps_per_epoch=len(dl_full1),
    epochs=max(FREEZE_EPOCHS,1), pct_start=min(PCT_START,0.5),
    div_factor=DIV_FACTOR, final_div_factor=FINAL_DIV
)

print("\n[Retrain-100%] Fase Freeze")
for ep in range(1, FREEZE_EPOCHS+1):
    tr_loss, tr_acc, tr_f1 = _train_epoch(dl_full1, opt, sched, freeze_bn=True,
                                          desc=f"FULL-FREEZE {ep}/{FREEZE_EPOCHS}", ema_obj=ema_full)
    print(f"[FULL-FREEZE {ep}/{FREEZE_EPOCHS}] loss={tr_loss:.4f} acc={tr_acc:.4f} f1={tr_f1:.4f}")

# Bersihkan memori sebelum FT (hindari fragmentasi/mandek)
gc.collect()
if DEVICE.type == "cuda":
    torch.cuda.empty_cache()

# ====== Fase 2: FULL FINETUNE (Unfreeze + LLRD; tetap tanpa validasi) ======
freeze_backbone(model_full, False)
opt = torch.optim.AdamW(param_groups_llrd(model_full))
sched = torch.optim.lr_scheduler.OneCycleLR(
    opt, max_lr=[pg['lr'] for pg in opt.param_groups], steps_per_epoch=len(dl_full2),
    epochs=max(EPOCHS-FREEZE_EPOCHS,1), pct_start=PCT_START,
    div_factor=DIV_FACTOR, final_div_factor=FINAL_DIV
)
# EMA saat FT mengikuti flag (default False untuk hemat VRAM)
ema_full = (ModelEMA(model_full, EMA_DECAY) if USE_EMA_FT else None)

print("[Retrain-100%] Fase Full Fine-Tune")
for ep in range(1, (EPOCHS-FREEZE_EPOCHS)+1):
    tr_loss, tr_acc, tr_f1 = _train_epoch(dl_full2, opt, sched, freeze_bn=(BATCH_SIZE<16),
                                          desc=f"FULL-FT {ep}/{EPOCHS-FREEZE_EPOCHS}", ema_obj=ema_full)
    lrs = _lr_list(opt)
    print(f"[FULL-FT {ep}/{EPOCHS-FREEZE_EPOCHS}] loss={tr_loss:.4f} acc={tr_acc:.4f} f1={tr_f1:.4f} | "
          f"LR s1={lrs[0]:.1e} s2={lrs[1]:.1e} s3={lrs[2]:.1e} s4={lrs[3]:.1e} head={lrs[4]:.1e}")

# ===== Simpan ckpt retrained (pakai EMA jika ada) =====
FULL_CKPT_PATH = RETRAIN_DIR / "retrained_full.pt"
torch.save({"model": (ema_full.ema if ema_full is not None else model_full).state_dict(),
            "classes": CLASSES_FULL}, FULL_CKPT_PATH)
print(f"[Retrain-100%] DONE. Saved: {FULL_CKPT_PATH}")

FULL_LAST_PATH = RETRAIN_DIR / "retrained_full_last.pt"
torch.save({"model": model_full.state_dict(), "classes": CLASSES_FULL}, FULL_LAST_PATH)
print(f"[Retrain-100%] LAST saved: {FULL_LAST_PATH}")


[Retrain-100%] Backbone: ConvNeXt-Small (convnext_small) | MODEL_ID=2

[Retrain-100%] Fase Freeze


FULL-FREEZE 1/1: 100%|██████████████████████████████████████████████████| 98/98 [05:25<00:00,  3.32s/it, acc=0.5070, los


[FULL-FREEZE 1/1] loss=1.2655 acc=0.5070 f1=0.2199
[Retrain-100%] Fase Full Fine-Tune


FULL-FT 1/9: 100%|██████████████████████████████████████████████████| 98/98 [08:45<00:00,  5.36s/it, acc=0.6881, loss=0.


[FULL-FT 1/9] loss=0.9887 acc=0.6881 f1=0.5469 | LR s1=2.4e-05 s2=3.7e-05 s3=6.1e-05 s4=1.2e-04 head=4.9e-04


...

[FULL-FT 7/9] loss=0.2706 acc=0.9981 f1=0.9982 | LR s1=7.1e-06 s2=1.1e-05 s3=1.8e-05 s4=3.5e-05 head=1.4e-04


FULL-FT 8/9: 100%|██████████████████████████████████████████████████| 98/98 [07:37<00:00,  4.66s/it, acc=0.9981, loss=0.


[FULL-FT 8/9] loss=0.2665 acc=0.9981 f1=0.9975 | LR s1=1.8e-06 s2=2.8e-06 s3=4.6e-06 s4=9.2e-06 head=3.7e-05


FULL-FT 9/9: 100%|██████████████████████████████████████████████████| 98/98 [07:36<00:00,  4.66s/it, acc=0.9974, loss=0.


[FULL-FT 9/9] loss=0.2684 acc=0.9974 f1=0.9973 | LR s1=2.2e-09 s2=3.3e-09 s3=5.5e-09 s4=1.1e-08 head=4.4e-08
[Retrain-100%] DONE. Saved: C:\Python\Template\Data Science\test\LOGIKA_UI\output\retrained_full\retrained_full.pt
[Retrain-100%] LAST saved: C:\Python\Template\Data Science\test\LOGIKA_UI\output\retrained_full\retrained_full_last.pt


In [19]:
# blok 8 — Prediksi Test ORI & buat submission.csv (robust, pilih model via CKPT_MODE)

import os, io, time, copy, warnings
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from tqdm import tqdm

# ====== Konfigurasi aman untuk test/infer ======
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = globals().get("BATCH_SIZE", 16)
IMG_SIZE_TRAIN2 = globals().get("IMG_SIZE_TRAIN2", 448)
CHANNELS_LAST = globals().get("CHANNELS_LAST", True)

# Matikan multiprocessing untuk mencegah "DataLoader worker exited unexpectedly"
NUM_WORKERS_TEST = 0
PIN_MEMORY = (DEVICE.type == "cuda")

DATA_ROOT    = Path("C:/Python/Template/Data Science/test/LOGIKA_UI")
TEST_ORI_DIR = DATA_ROOT / "Test/Test"
SAMPLE_SUB   = DATA_ROOT / "sample_submission.csv"
OUT_CSV      = Path("C:/Python/Template/Data Science/test/LOGIKA_UI/output/submission.csv")

OUT_DIR     = Path("C:/Python/Template/Data Science/test/LOGIKA_UI/output")
CKPT_DIR    = OUT_DIR / "checkpoints"
RETRAIN_DIR = OUT_DIR / "retrained_full"   # <-- diperbaiki (bukan folder 'retrained_full_last')

# ====== Pilih model yang dipakai ======
# Opsi: "full_best" (retrained_full.pt), "full_last" (retrained_full_last.pt),
#       "best" (best.pt), "last" (last.pt), "manual"
CKPT_MODE = "retrain_full_last"   # ganti di sini sesuai kebutuhan
MANUAL_CKPT_PATH = ""     # isi path file .pt jika CKPT_MODE="manual"

def _resolve_ckpt():
    # Prioritas berdasarkan CKPT_MODE
    mode = CKPT_MODE.lower().strip()
    candidates = []
    # dari variabel global (kalau ada)
    if "FULL_CKPT_PATH" in globals(): candidates.append(Path(FULL_CKPT_PATH))
    if "BEST_CKPT_PATH" in globals(): candidates.append(Path(BEST_CKPT_PATH))
    if "LAST_CKPT_PATH" in globals(): candidates.append(Path(LAST_CKPT_PATH))

    mapping = {
        "full_best": RETRAIN_DIR / "retrained_full.pt",
        "full_last": RETRAIN_DIR / "retrained_full_last.pt",
        "best":      CKPT_DIR / "best.pt",
        "last":      CKPT_DIR / "last.pt",
    }

    if mode == "manual":
        p = Path(MANUAL_CKPT_PATH)
        return p if p.exists() else None

    # spesifik mode
    if mode in mapping:
        p = mapping[mode]
        return p if p.exists() else None

    # "auto" atau fallback: coba berurutan yang paling kuat
    ordered = [
        RETRAIN_DIR / "retrained_full.pt",
        RETRAIN_DIR / "retrained_full_last.pt",
        CKPT_DIR / "best.pt",
        CKPT_DIR / "last.pt",
    ] + candidates

    for p in ordered:
        if p is not None and Path(p).exists():
            return Path(p)
    return None

def build_eval_tf(size:int):
    return transforms.Compose([
        transforms.Resize(int(size*1.14), interpolation=InterpolationMode.BICUBIC),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
    ])

# ====== Dataset tanpa label yang robust ======
class TestImageFolderNoLabel(Dataset):
    def __init__(self, root, size=IMG_SIZE_TRAIN2):
        self.root = Path(root)
        self.fns = sorted([p for p in self.root.glob("*") if p.suffix.lower() in (".jpg",".jpeg",".png",".bmp",".tif",".tiff")])
        self.tf  = build_eval_tf(size)
        self.size = size

    def __len__(self): 
        return len(self.fns)

    def _safe_open(self, p: Path):
        # Coba beberapa cara agar tidak crash kalau file bermasalah
        try:
            return Image.open(p).convert("RGB")
        except Exception:
            try:
                with open(p, "rb") as f:
                    return Image.open(io.BytesIO(f.read())).convert("RGB")
            except Exception:
                # fallback: gambar abu-abu agar tetap ada output
                return Image.new("RGB", (self.size, self.size), (128,128,128))

    def __getitem__(self, i):
        p = self.fns[i]
        img = self._safe_open(p)
        return self.tf(img), p.name

def make_loader(ds, batch, shuffle=False):
    return DataLoader(
        ds, batch_size=batch, shuffle=shuffle, pin_memory=PIN_MEMORY,
        num_workers=NUM_WORKERS_TEST, persistent_workers=False
    )

# ====== Fallback model defs bila belum ada di sesi ======
if "GeM" not in globals():
    class GeM(nn.Module):
        def __init__(self, p=3.0, eps=1e-6, learn_p=True):
            super().__init__()
            self.p = nn.Parameter(torch.ones(1)*p) if learn_p else torch.tensor([p])
            self.eps = eps
        def forward(self, x):
            x = x.clamp(min=self.eps).pow(self.p)
            x = F.avg_pool2d(x, (x.size(-2), x.size(-1)))
            return x.pow(1./self.p)

if "ClsNet" not in globals():
    import timm
    class ClsNet(nn.Module):
        def __init__(self, num_classes, pretrained=False):
            super().__init__()
            self.backbone = timm.create_model("convnext_small", pretrained=pretrained, features_only=False, num_classes=0, global_pool="")
            self.pool = GeM(p=3.0, learn_p=True)
            self.bnneck = nn.BatchNorm1d(self.backbone.num_features, eps=1e-5, momentum=0.1)
            self.bnneck.bias.requires_grad_(False)
            self.head = nn.Linear(self.backbone.num_features, num_classes, bias=True)
        def forward(self, x):
            f = self.backbone(x)
            f = self.pool(f).flatten(1)
            f = self.bnneck(f)
            return self.head(f)

# ====== Resolve checkpoint ======
ckpt_path = _resolve_ckpt()
assert ckpt_path is not None and ckpt_path.exists(), (
    f"Checkpoint tidak ditemukan untuk CKPT_MODE='{CKPT_MODE}'.\n"
    f"Coba set salah satu:\n"
    f"  CKPT_MODE='full_best'  -> {RETRAIN_DIR/'retrained_full.pt'}\n"
    f"  CKPT_MODE='full_last'  -> {RETRAIN_DIR/'retrained_full_last.pt'}\n"
    f"  CKPT_MODE='best'       -> {CKPT_DIR/'best.pt'}\n"
    f"  CKPT_MODE='last'       -> {CKPT_DIR/'last.pt'}\n"
    f"  CKPT_MODE='manual' + MANUAL_CKPT_PATH='.../model.pt'\n"
)
print(f"[Predict] Using checkpoint ({CKPT_MODE}): {ckpt_path}")

# ====== Load model ======
ck = torch.load(ckpt_path, map_location=DEVICE)
classes = ck["classes"]; num_classes = len(classes)

model = ClsNet(num_classes=num_classes, pretrained=False).to(DEVICE)
model.load_state_dict(ck["model"], strict=True)
model.eval()
if CHANNELS_LAST and DEVICE.type == "cuda":
    model = model.to(memory_format=torch.channels_last)

# ====== Loader test ori ======
ds = TestImageFolderNoLabel(TEST_ORI_DIR, size=IMG_SIZE_TRAIN2)
assert len(ds) > 0, f"Tidak ada gambar di {TEST_ORI_DIR}"
dl = make_loader(ds, BATCH_SIZE*2, shuffle=False)

# ====== Prediksi ======
def _bar(desc): return dict(desc=desc, ncols=120, bar_format="{l_bar}{bar:50}{r_bar}")

ids, preds = [], []
pbar = tqdm(dl, **_bar("Predict(Test ORI)"))
with torch.no_grad():
    for imgs, names in pbar:
        imgs = imgs.to(DEVICE, non_blocking=True)
        if CHANNELS_LAST and DEVICE.type == "cuda":
            imgs = imgs.to(memory_format=torch.channels_last)
        with autocast():
            logits = model(imgs)
        y = logits.argmax(1).cpu().numpy().tolist()
        ids.extend([Path(n).stem for n in names])
        preds.extend([classes[i] for i in y])
pbar.close()

# ====== Susun submission ======
sub = pd.read_csv(SAMPLE_SUB)
sub_ids = sub["id"].astype(str)

# map by stem
pred_map_stem = dict(zip(ids, preds))
sub["style"] = sub_ids.map(pred_map_stem)

# jika masih NaN (format id berbeda), coba cocokkan dengan nama file ber-ekstensi
if sub["style"].isna().any():
    ext_map = {f"{k}.jpg": v for k,v in pred_map_stem.items()}
    ext_map.update({f"{k}.png": v for k,v in pred_map_stem.items()})
    sub["style"] = sub["style"].combine_first(sub_ids.map(ext_map))

# fallback terakhir: isi modus prediksi
if sub["style"].isna().any():
    fallback = pd.Series(preds).mode().iloc[0] if len(preds) else classes[0]
    sub["style"] = sub["style"].fillna(fallback)

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
sub.to_csv(OUT_CSV, index=False)
print(f"[Submission] saved to: {OUT_CSV}")


[Predict] Using checkpoint (retrain_full_last): C:\Python\Template\Data Science\test\LOGIKA_UI\output\retrained_full\retrained_full.pt


Predict(Test ORI): 100%|██████████████████████████████████████████████████| 14/14 [01:45<00:00,  7.56s/it]

[Submission] saved to: C:\Python\Template\Data Science\test\LOGIKA_UI\output\submission.csv


In [3]:
# blok 8 — Prediksi final (TTA + baseline blend + proto re-rank) & submission.csv
# Patch aman RTX 3050:
# - Dedup TTA scales
# - Dynamic micro-batch per shape (512→<=8, 448→<=12)
# - FiveCrop dimatikan untuk resolusi tinggi (>=448)
# - AMP universal (torch.amp.autocast)
# Catatan: Bagian kalibrasi hardcode dihapus sesuai permintaan.

import os, io, math, copy, warnings
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import amp
from torch.cuda.amp import GradScaler
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.transforms import InterpolationMode, functional as TF
from tqdm import tqdm

# ====== Konfigurasi inferensi ======
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = globals().get("BATCH_SIZE", 16)
IMG_SIZE   = globals().get("IMG_SIZE_TRAIN2", 384)
CHANNELS_LAST = globals().get("CHANNELS_LAST", True)

DATA_ROOT    = Path("C:/Python/Template/Data Science/test/LOGIKA_UI")
TEST_ORI_DIR = DATA_ROOT / "Test/Test"
SAMPLE_SUB   = DATA_ROOT / "sample_submission.csv"
OUT_CSV      = Path("C:/Python/Template/Data Science/test/LOGIKA_UI/output/submission3.csv")

# ====== Checkpoint ======
CKPT_MODE = "manual"
MANUAL_CKPT_PATH = "C:/Python/Template/Data Science/test/LOGIKA_UI/output/retrained_full/retrained_full.pt"

# ====== Pipeline (sesuai tuning, dengan patch keamanan)
USE_TTA         = True
# Deduplicate scales agar 384 tidak dobel jika IMG_SIZE=384
TTA_SCALES      = sorted(set([384, IMG_SIZE, 512]))
TTA_FIVE_CROP   = True     # akan dimatikan otomatis untuk size >= 448 (lihat _use_five_crop)
TTA_HFLIP       = True
TTA_MAX_BATCH   = max(32, BATCH_SIZE * 2)  # batas atas; akan di-cap per shape

USE_BASELINE_BLEND = True
TTA_WEIGHT         = 0.70  # 0.7*TTA + 0.3 baseline

USE_PROTO_RERANK = True
PROTO_LAMBDA     = 0.50
COSINE_TEMP      = 10.0

# ====== Util ======
def _resolve_ckpt():
    p = Path(MANUAL_CKPT_PATH) if CKPT_MODE.lower() == "manual" else None
    return p if p and p.exists() else None

class TestImageFolderNoLabel(Dataset):
    def __init__(self, root):
        self.root = Path(root)
        exts = (".jpg",".jpeg",".png",".bmp",".tif",".tiff")
        self.fns = sorted([p for p in self.root.glob("*") if p.suffix.lower() in exts])
    def __len__(self): return len(self.fns)
    def __getitem__(self, i): return self.fns[i]

def safe_open(p: Path, fallback_size=IMG_SIZE):
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        try:
            with open(p, "rb") as f:
                return Image.open(io.BytesIO(f.read())).convert("RGB")
        except Exception:
            return Image.new("RGB", (fallback_size, fallback_size), (128,128,128))

# ====== Model ======
class GeM(nn.Module):
    def __init__(self, p=3.0, eps=1e-6, learn_p=True):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1)*p) if learn_p else torch.tensor([p])
        self.eps = eps
    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        x = F.avg_pool2d(x, (x.size(-2), x.size(-1)))
        return x.pow(1./self.p)

class ClsNet(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        import timm
        self.backbone = timm.create_model(
            "convnext_small", pretrained=pretrained,
            features_only=False, num_classes=0, global_pool=""
        )
        self.pool = GeM(p=3.0, learn_p=True)
        self.bnneck = nn.BatchNorm1d(self.backbone.num_features, eps=1e-5, momentum=0.1)
        self.bnneck.bias.requires_grad_(False)
        self.head = nn.Linear(self.backbone.num_features, num_classes, bias=True)
    def forward_feats(self, x):
        f = self.backbone(x)
        f = self.pool(f).flatten(1)
        f = self.bnneck(f)
        return f
    def forward_from_feats(self, f): return self.head(f)
    def forward(self, x): return self.forward_from_feats(self.forward_feats(x))

# ====== Load ckpt & classes ======
ckpt_path = _resolve_ckpt()
assert ckpt_path is not None and ckpt_path.exists(), f"Checkpoint tidak ditemukan: {ckpt_path}"
print(f"[Predict] Using checkpoint: {ckpt_path}")

ck = torch.load(ckpt_path, map_location=DEVICE)
state_dict = ck.get("model") or ck.get("state_dict") or ck
classes = ck.get("classes")
if classes is None:
    ss = pd.read_csv(SAMPLE_SUB)
    u = ss["style"].dropna().unique().tolist()
    classes = sorted(u) if len(u) else None
    if classes is None:
        # Infer jumlah kelas dari checkpoint (head.weight)
        head_key = None
        for k in ("head.weight", "module.head.weight"):
            if k in state_dict:
                head_key = k; break
        if head_key is None:
            raise RuntimeError("Tidak bisa menginfer classes: checkpoint tak punya 'classes' dan sample_submission kosong.")
        C_guess = int(state_dict[head_key].shape[0])
        classes = [f"class_{i}" for i in range(C_guess)]

C = len(classes); cls2idx = {c:i for i,c in enumerate(classes)}

model = ClsNet(num_classes=C, pretrained=False).to(DEVICE)
# hapus prefix 'module.' jika ada
if any(k.startswith("module.") for k in state_dict.keys()):
    state_dict = {k.replace("module.", "", 1): v for k,v in state_dict.items()}
model.load_state_dict(state_dict, strict=True)
model.eval()
if CHANNELS_LAST and DEVICE.type == "cuda":
    model = model.to(memory_format=torch.channels_last)

# ====== TTA utilities (bucket by shape) ======
_MEAN = (0.485, 0.456, 0.406); _STD = (0.229, 0.224, 0.225)
def _pre_resize(img, size): return transforms.Resize(int(size*1.14), interpolation=InterpolationMode.BICUBIC)(img)
to_tensor_norm = transforms.Compose([transforms.ToTensor(), transforms.Normalize(_MEAN, _STD)])

def _use_five_crop(sz:int)->bool:
    # Patch: untuk resolusi tinggi, pakai CenterCrop saja (hemat memori)
    return TTA_FIVE_CROP and (sz < 448)

def _crop_variants(img, size, five_crop=True):
    return list(transforms.FiveCrop(size)(img)) if five_crop else [transforms.CenterCrop(size)(img)]

def gen_tta_variants(img):
    variants = []
    if USE_TTA:
        for sz in TTA_SCALES:
            base = _pre_resize(img, sz)
            crops = _crop_variants(base, sz, five_crop=_use_five_crop(sz))
            for c in crops: variants.append((to_tensor_norm(c),(sz,sz)))
            if TTA_HFLIP:
                for c in crops: variants.append((to_tensor_norm(TF.hflip(c)),(sz,sz)))
    else:
        sz = IMG_SIZE
        base = _pre_resize(img, sz); c = transforms.CenterCrop(sz)(base)
        variants.append((to_tensor_norm(c),(sz,sz)))
    return variants

def gen_single_variant(img):
    sz = IMG_SIZE
    base = _pre_resize(img, sz); c = transforms.CenterCrop(sz)(base)
    return [(to_tensor_norm(c),(sz,sz))]

def _batch_cap_for_shape(shp, base=IMG_SIZE, max_batch=TTA_MAX_BATCH):
    h, w = shp
    # skala berbasis luas piksel dibanding base
    scale = (base * base) / max(h*w, 1)
    b = max(1, int(scale * max_batch))
    # cap khusus resolusi tinggi untuk aman VRAM
    if max(h, w) >= 512:
        b = min(b, 8)    # 4–8 sangat aman di 4GB
    elif max(h, w) >= 448:
        b = min(b, 12)
    else:
        b = min(b, max_batch)
    return max(1, b)

@torch.no_grad()
def infer_bucketed(paths, make_variants):
    N, D = len(paths), model.bnneck.num_features
    feat_sums  = torch.zeros((N, D), dtype=torch.float32)
    logit_sums = torch.zeros((N, C), dtype=torch.float32)
    counts     = torch.zeros(N, dtype=torch.float32)

    owners, tensors, shapes = [], [], []
    for i, p in enumerate(paths):
        img = safe_open(p)
        for t, shp in make_variants(img):
            owners.append(i); tensors.append(t); shapes.append(shp)

    from collections import defaultdict
    buckets = defaultdict(list)
    for idx, shp in enumerate(shapes): buckets[shp].append(idx)

    model.eval()
    for shp, idxs in buckets.items():
        start = 0
        bcap = _batch_cap_for_shape(shp)
        while start < len(idxs):
            bidx = idxs[start:start+bcap]
            batch = torch.stack([tensors[k] for k in bidx], 0).to(DEVICE, non_blocking=True)
            if CHANNELS_LAST and DEVICE.type == "cuda":
                batch = batch.to(memory_format=torch.channels_last)
            with amp.autocast(device_type=DEVICE.type):
                feats = model.forward_feats(batch)
                logits = model.forward_from_feats(feats)
            feats = feats.detach().cpu(); logits = logits.detach().cpu()
            for j, k in enumerate(bidx):
                o = owners[k]
                feat_sums[o]  += feats[j]
                logit_sums[o] += logits[j]
                counts[o]     += 1.0
            start += bcap
            if DEVICE.type == "cuda":
                torch.cuda.empty_cache()

    counts = counts.clamp_min(1.0).view(-1,1)
    return feat_sums / counts, logit_sums / counts

# ====== Infer (kumpulkan logits & feats) ======
ds = TestImageFolderNoLabel(TEST_ORI_DIR)
assert len(ds) > 0, f"Tidak ada gambar di {TEST_ORI_DIR}"

ids, FEATS_TTA, LOGITS_TTA = [], [], []
FEATS_BASE, LOGITS_BASE = [], []

pbar = tqdm(range(0, len(ds), BATCH_SIZE), ncols=120, bar_format="{l_bar}{bar:50}{r_bar}", desc="Predict")
with torch.no_grad():
    for start in pbar:
        end = min(start + BATCH_SIZE, len(ds))
        paths = [ds[i] for i in range(start, end)]
        f_tta, z_tta = infer_bucketed(paths, gen_tta_variants)
        FEATS_TTA.append(f_tta); LOGITS_TTA.append(z_tta)
        f_b, z_b = infer_bucketed(paths, gen_single_variant)
        FEATS_BASE.append(f_b); LOGITS_BASE.append(z_b)
        ids.extend([p.stem for p in paths])
pbar.close()

FEATS_TTA  = torch.cat(FEATS_TTA,  0)
LOGITS_TTA = torch.cat(LOGITS_TTA, 0)
FEATS_BASE = torch.cat(FEATS_BASE, 0)
LOGITS_BASE= torch.cat(LOGITS_BASE,0)

# ====== Blending ======
feats  = (1.0 - TTA_WEIGHT) * FEATS_BASE  + TTA_WEIGHT * FEATS_TTA if USE_BASELINE_BLEND else FEATS_TTA
logits = (1.0 - TTA_WEIGHT) * LOGITS_BASE + TTA_WEIGHT * LOGITS_TTA if USE_BASELINE_BLEND else LOGITS_TTA

# ====== Prototypical re-ranking ======
if USE_PROTO_RERANK:
    W = model.head.weight.detach().cpu()      # (C,D)
    f_norm = F.normalize(feats, dim=1)
    W_norm = F.normalize(W,     dim=1)
    cos = torch.matmul(f_norm, W_norm.T)      # (N,C)
    logits = logits + PROTO_LAMBDA * (COSINE_TEMP * cos)

# ====== Prob & pred ======
probs = logits.float().softmax(dim=1)
pred_idx = probs.argmax(dim=1).tolist()
preds = [classes[i] for i in pred_idx]

# ====== Susun submission ======
sub = pd.read_csv(SAMPLE_SUB)
sub_ids = sub["id"].astype(str)
pred_map_stem = dict(zip(ids, preds))
sub["style"] = sub_ids.map(pred_map_stem)

# fallback: coba id dengan ekstensi
if sub["style"].isna().any():
    ext_map = {f"{k}.jpg": v for k,v in pred_map_stem.items()}
    ext_map.update({f"{k}.png": v for k,v in pred_map_stem.items()})
    sub["style"] = sub["style"].combine_first(sub_ids.map(ext_map))

# fallback terakhir: modus
if sub["style"].isna().any():
    fallback = pd.Series(preds).mode().iloc[0] if len(preds) else classes[0]
    sub["style"] = sub["style"].fillna(fallback)

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
sub.to_csv(OUT_CSV, index=False)
print(f"[Submission] saved to: {OUT_CSV}")


[Predict] Using checkpoint: C:\Python\Template\Data Science\test\LOGIKA_UI\output\retrained_full\retrained_full.pt


Predict: 100%|██████████████████████████████████████████████████| 28/28 [06:31<00:00, 13.96s/it]

[Submission] saved to: C:\Python\Template\Data Science\test\LOGIKA_UI\output\submission3.csv


In [1]:
# blok 8 — Prediksi final (TTA + baseline blend + proto re-rank + calibration) & submission.csv
# Patch aman RTX 3050:
# - Dedup TTA scales
# - Dynamic micro-batch per shape (512→<=8, 448→<=12)
# - FiveCrop dimatikan untuk resolusi tinggi (>=448)
# - AMP universal (torch.amp.autocast)

import os, io, math, copy, warnings
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import amp
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.transforms import InterpolationMode, functional as TF
from tqdm import tqdm

# ====== Konfigurasi inferensi ======
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = globals().get("BATCH_SIZE", 16)
IMG_SIZE   = globals().get("IMG_SIZE_TRAIN2", 384)
CHANNELS_LAST = globals().get("CHANNELS_LAST", True)

DATA_ROOT    = Path("C:/Python/Template/Data Science/test/LOGIKA_UI")
TEST_ORI_DIR = DATA_ROOT / "Test/Test"
SAMPLE_SUB   = DATA_ROOT / "sample_submission.csv"
OUT_CSV      = Path("C:/Python/Template/Data Science/test/LOGIKA_UI/output/submission2.csv")

# ====== Checkpoint ======
CKPT_MODE = "manual"
MANUAL_CKPT_PATH = "C:/Python/Template/Data Science/test/LOGIKA_UI/output/retrained_full/retrained_full.pt"

# ====== Pipeline (sesuai tuning, dengan patch keamanan)
USE_TTA         = True
# Deduplicate scales agar 384 tidak dobel jika IMG_SIZE=384
TTA_SCALES      = sorted(set([384, IMG_SIZE, 512]))
TTA_FIVE_CROP   = True     # akan dimatikan otomatis untuk size >= 448 (lihat _use_five_crop)
TTA_HFLIP       = True
TTA_MAX_BATCH   = max(32, BATCH_SIZE * 2)  # batas atas; akan di-cap per shape

USE_BASELINE_BLEND = True
TTA_WEIGHT         = 0.70  # 0.7*TTA + 0.3 baseline

USE_PROTO_RERANK = True
PROTO_LAMBDA     = 0.50
COSINE_TEMP      = 10.0

# ====== Kalibrasi (hardcode dari hasil tuning) ======
CALIB_CLASSES = ["balinese","batak","dayak","javanese","minangkabau"]
CALIB_SCALE = {
    "balinese": 1.4169902801513672,
    "batak": 1.7252342700958252,
    "dayak": 3.1149961948394775,
    "javanese": 1.5015865564346313,
    "minangkabau": 1.3272558450698853
}
CALIB_BIAS = {
    "balinese": -0.02708914317190647,
    "batak": -0.3606128692626953,
    "dayak": 0.0019426785875111818,
    "javanese": 0.5581263899803162,
    "minangkabau": -0.3879375457763672
}

# ====== Util ======
def _resolve_ckpt():
    p = Path(MANUAL_CKPT_PATH) if CKPT_MODE.lower() == "manual" else None
    return p if p and p.exists() else None

class TestImageFolderNoLabel(Dataset):
    def __init__(self, root):
        self.root = Path(root)
        exts = (".jpg",".jpeg",".png",".bmp",".tif",".tiff")
        self.fns = sorted([p for p in self.root.glob("*") if p.suffix.lower() in exts])
    def __len__(self): return len(self.fns)
    def __getitem__(self, i): return self.fns[i]

def safe_open(p: Path, fallback_size=IMG_SIZE):
    try:
        return Image.open(p).convert("RGB")
    except Exception:
        try:
            with open(p, "rb") as f:
                return Image.open(io.BytesIO(f.read())).convert("RGB")
        except Exception:
            return Image.new("RGB", (fallback_size, fallback_size), (128,128,128))

# ====== Model ======
class GeM(nn.Module):
    def __init__(self, p=3.0, eps=1e-6, learn_p=True):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1)*p) if learn_p else torch.tensor([p])
        self.eps = eps
    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        x = F.avg_pool2d(x, (x.size(-2), x.size(-1)))
        return x.pow(1./self.p)

class ClsNet(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        import timm
        self.backbone = timm.create_model(
            "convnext_small", pretrained=pretrained,
            features_only=False, num_classes=0, global_pool=""
        )
        self.pool = GeM(p=3.0, learn_p=True)
        self.bnneck = nn.BatchNorm1d(self.backbone.num_features, eps=1e-5, momentum=0.1)
        self.bnneck.bias.requires_grad_(False)
        self.head = nn.Linear(self.backbone.num_features, num_classes, bias=True)
    def forward_feats(self, x):
        f = self.backbone(x)
        f = self.pool(f).flatten(1)
        f = self.bnneck(f)
        return f
    def forward_from_feats(self, f): return self.head(f)
    def forward(self, x): return self.forward_from_feats(self.forward_feats(x))

# ====== Load ckpt & classes ======
ckpt_path = _resolve_ckpt()
assert ckpt_path is not None and ckpt_path.exists(), f"Checkpoint tidak ditemukan: {ckpt_path}"
print(f"[Predict] Using checkpoint: {ckpt_path}")

ck = torch.load(ckpt_path, map_location=DEVICE)
state_dict = ck.get("model") or ck.get("state_dict") or ck
classes = ck.get("classes")
if classes is None:
    ss = pd.read_csv(SAMPLE_SUB)
    u = ss["style"].dropna().unique().tolist()
    classes = sorted(u) if len(u) else CALIB_CLASSES[:]

C = len(classes); cls2idx = {c:i for i,c in enumerate(classes)}

model = ClsNet(num_classes=C, pretrained=False).to(DEVICE)
# hapus prefix 'module.' jika ada
if any(k.startswith("module.") for k in state_dict.keys()):
    state_dict = {k.replace("module.", "", 1): v for k,v in state_dict.items()}
model.load_state_dict(state_dict, strict=True)
model.eval()
if CHANNELS_LAST and DEVICE.type == "cuda":
    model = model.to(memory_format=torch.channels_last)

# ====== TTA utilities (bucket by shape) ======
_MEAN = (0.485, 0.456, 0.406); _STD = (0.229, 0.224, 0.225)
def _pre_resize(img, size): return transforms.Resize(int(size*1.14), interpolation=InterpolationMode.BICUBIC)(img)
to_tensor_norm = transforms.Compose([transforms.ToTensor(), transforms.Normalize(_MEAN, _STD)])

def _use_five_crop(sz:int)->bool:
    # Patch: untuk resolusi tinggi, pakai CenterCrop saja (hemat memori)
    return TTA_FIVE_CROP and (sz < 448)

def _crop_variants(img, size, five_crop=True):
    return list(transforms.FiveCrop(size)(img)) if five_crop else [transforms.CenterCrop(size)(img)]

def gen_tta_variants(img):
    variants = []
    if USE_TTA:
        for sz in TTA_SCALES:
            base = _pre_resize(img, sz)
            crops = _crop_variants(base, sz, five_crop=_use_five_crop(sz))
            for c in crops: variants.append((to_tensor_norm(c),(sz,sz)))
            if TTA_HFLIP:
                for c in crops: variants.append((to_tensor_norm(TF.hflip(c)),(sz,sz)))
    else:
        sz = IMG_SIZE
        base = _pre_resize(img, sz); c = transforms.CenterCrop(sz)(base)
        variants.append((to_tensor_norm(c),(sz,sz)))
    return variants

def gen_single_variant(img):
    sz = IMG_SIZE
    base = _pre_resize(img, sz); c = transforms.CenterCrop(sz)(base)
    return [(to_tensor_norm(c),(sz,sz))]

def _batch_cap_for_shape(shp, base=IMG_SIZE, max_batch=TTA_MAX_BATCH):
    h, w = shp
    # skala berbasis luas piksel dibanding base
    scale = (base * base) / max(h*w, 1)
    b = max(1, int(scale * max_batch))
    # cap khusus resolusi tinggi untuk aman VRAM
    if max(h, w) >= 512:
        b = min(b, 8)    # 4–8 sangat aman di 4GB
    elif max(h, w) >= 448:
        b = min(b, 12)
    else:
        b = min(b, max_batch)
    return max(1, b)

@torch.no_grad()
def infer_bucketed(paths, make_variants):
    N, D = len(paths), model.bnneck.num_features
    feat_sums  = torch.zeros((N, D), dtype=torch.float32)
    logit_sums = torch.zeros((N, C), dtype=torch.float32)
    counts     = torch.zeros(N, dtype=torch.float32)

    owners, tensors, shapes = [], [], []
    for i, p in enumerate(paths):
        img = safe_open(p)
        for t, shp in make_variants(img):
            owners.append(i); tensors.append(t); shapes.append(shp)

    from collections import defaultdict
    buckets = defaultdict(list)
    for idx, shp in enumerate(shapes): buckets[shp].append(idx)

    model.eval()
    for shp, idxs in buckets.items():
        start = 0
        bcap = _batch_cap_for_shape(shp)
        while start < len(idxs):
            bidx = idxs[start:start+bcap]
            batch = torch.stack([tensors[k] for k in bidx], 0).to(DEVICE, non_blocking=True)
            if CHANNELS_LAST and DEVICE.type == "cuda":
                batch = batch.to(memory_format=torch.channels_last)
            with amp.autocast(device_type=DEVICE.type):
                feats = model.forward_feats(batch)
                logits = model.forward_from_feats(feats)
            feats = feats.detach().cpu(); logits = logits.detach().cpu()
            for j, k in enumerate(bidx):
                o = owners[k]
                feat_sums[o]  += feats[j]
                logit_sums[o] += logits[j]
                counts[o]     += 1.0
            start += bcap
            # (opsional) jika sangat mepet, kosongkan cache
            if DEVICE.type == "cuda":
                torch.cuda.empty_cache()

    counts = counts.clamp_min(1.0).view(-1,1)
    return feat_sums / counts, logit_sums / counts

# ====== Infer (kumpulkan logits & feats) ======
ds = TestImageFolderNoLabel(TEST_ORI_DIR)
assert len(ds) > 0, f"Tidak ada gambar di {TEST_ORI_DIR}"

ids, FEATS_TTA, LOGITS_TTA = [], [], []
FEATS_BASE, LOGITS_BASE = [], []

pbar = tqdm(range(0, len(ds), BATCH_SIZE), ncols=120, bar_format="{l_bar}{bar:50}{r_bar}", desc="Predict")
with torch.no_grad():
    for start in pbar:
        end = min(start + BATCH_SIZE, len(ds))
        paths = [ds[i] for i in range(start, end)]
        f_tta, z_tta = infer_bucketed(paths, gen_tta_variants)
        FEATS_TTA.append(f_tta); LOGITS_TTA.append(z_tta)
        f_b, z_b = infer_bucketed(paths, gen_single_variant)
        FEATS_BASE.append(f_b); LOGITS_BASE.append(z_b)
        ids.extend([p.stem for p in paths])
pbar.close()

FEATS_TTA  = torch.cat(FEATS_TTA,  0)
LOGITS_TTA = torch.cat(LOGITS_TTA, 0)
FEATS_BASE = torch.cat(FEATS_BASE, 0)
LOGITS_BASE= torch.cat(LOGITS_BASE,0)

# ====== Blending ======
feats  = (1.0 - TTA_WEIGHT) * FEATS_BASE  + TTA_WEIGHT * FEATS_TTA if USE_BASELINE_BLEND else FEATS_TTA
logits = (1.0 - TTA_WEIGHT) * LOGITS_BASE + TTA_WEIGHT * LOGITS_TTA if USE_BASELINE_BLEND else LOGITS_TTA

# ====== Prototypical re-ranking ======
if USE_PROTO_RERANK:
    W = model.head.weight.detach().cpu()      # (C,D)
    f_norm = F.normalize(feats, dim=1)
    W_norm = F.normalize(W,     dim=1)
    cos = torch.matmul(f_norm, W_norm.T)      # (N,C)
    logits = logits + PROTO_LAMBDA * (COSINE_TEMP * cos)

# ====== Terapkan kalibrasi per-kelas (vector scaling + bias) ======
scale_vec = torch.ones(C, dtype=torch.float32)
bias_vec  = torch.zeros(C, dtype=torch.float32)
for c_name, idx in cls2idx.items():
    s = CALIB_SCALE.get(c_name, 1.0)
    b = CALIB_BIAS.get(c_name, 0.0)
    scale_vec[idx] = float(s)
    bias_vec[idx]  = float(b)

logits = logits * scale_vec.view(1, C) + bias_vec.view(1, C)

# ====== Prob & pred ======
probs = logits.float().softmax(dim=1)
pred_idx = probs.argmax(dim=1).tolist()
preds = [classes[i] for i in pred_idx]

# ====== Susun submission ======
sub = pd.read_csv(SAMPLE_SUB)
sub_ids = sub["id"].astype(str)
pred_map_stem = dict(zip(ids, preds))
sub["style"] = sub_ids.map(pred_map_stem)

# fallback: coba id dengan ekstensi
if sub["style"].isna().any():
    ext_map = {f"{k}.jpg": v for k,v in pred_map_stem.items()}
    ext_map.update({f"{k}.png": v for k,v in pred_map_stem.items()})
    sub["style"] = sub["style"].combine_first(sub_ids.map(ext_map))

# fallback terakhir: modus
if sub["style"].isna().any():
    fallback = pd.Series(preds).mode().iloc[0] if len(preds) else classes[0]
    sub["style"] = sub["style"].fillna(fallback)

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
sub.to_csv(OUT_CSV, index=False)
print(f"[Submission] saved to: {OUT_CSV}")


[Predict] Using checkpoint: C:\Python\Template\Data Science\test\LOGIKA_UI\output\retrained_full\retrained_full.pt


c:\Python\Template\Data Science\test\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Predict: 100%|██████████████████████████████████████████████████| 28/28 [05:18<00:00, 11.37s/it]

[Submission] saved to: C:\Python\Template\Data Science\test\LOGIKA_UI\output\submission2.csv


In [4]:
# =========================================================
# CEK PERBEDAAN 'style' TIGA CSV (ID SAMA / TIDAK SAMA) — ROBUST
# =========================================================
# FILE1 = r"C:\Python\Template\Tugas_Alpro_II\LogikaUI_2025\submission (31)  (best).csv"
FILE1 = r"C:\Python\Template\Data Science\test\LOGIKA_UI\output\submission3.csv"
FILE2 = r"C:\Python\Template\Data Science\test\LOGIKA_UI\output\submission3.csv"
FILE3 = r"C:\Python\Template\Tugas_Alpro_II\LogikaUI_2025\kunci_jawaban.csv"

ID_COL    = "id"
VALUE_COL = "style"

# Normalisasi ringan agar perbandingan adil
STRIP_WHITESPACE = True   # hapus spasi kiri/kanan
CASE_SENSITIVE   = True   # False kalau mau abaikan kapitalisasi

import pandas as pd

def _normalize(series: pd.Series) -> pd.Series:
    s = series.astype("string")
    if STRIP_WHITESPACE:
        s = s.str.strip()
    if not CASE_SENSITIVE:
        s = s.str.lower()
    return s

def load_series(path: str, id_col: str, val_col: str) -> pd.Series:
    """
    Baca dua kolom (id, style) sebagai Series bertipe string,
    buang duplikat id (pakai kemunculan pertama), dan normalisasi.
    """
    df = pd.read_csv(
        path,
        usecols=[id_col, val_col],
        dtype={id_col: "string"},
        keep_default_na=False,  # cegah "" jadi NaN; biarkan benar-benar string kosong
    )
    # drop duplicate ids (keep first)
    if df[id_col].duplicated(keep=False).any():
        ndup = df[id_col].duplicated(keep=False).sum()
        print(f"[PERINGATAN] {ndup} baris duplikat '{id_col}' di {path}. "
              f"Menggunakan kemunculan pertama per id.")
        df = df.drop_duplicates(subset=[id_col], keep="first")

    df[val_col] = _normalize(df[val_col])
    s = df.set_index(id_col)[val_col]
    s.name = path  # nama kolom pakai path agar jelas
    return s

def main():
    s1 = load_series(FILE1, ID_COL, VALUE_COL)
    s2 = load_series(FILE2, ID_COL, VALUE_COL)
    s3 = load_series(FILE3, ID_COL, VALUE_COL)

    # Gabungkan semua ID (outer) untuk melihat ketidaksinkronan
    all_df = pd.concat([s1.rename("FILE1"), s2.rename("FILE2"), s3.rename("FILE3")],
                       axis=1, join="outer")

    print("=== RINGKASAN ===")
    print(f"- Total baris FILE1: {len(s1)}")
    print(f"- Total baris FILE2: {len(s2)}")
    print(f"- Total baris FILE3: {len(s3)}")

    # ID yang tidak sinkron (ada NaN di salah satu kolom)
    not_sync = all_df[all_df.isna().any(axis=1)]
    print(f"- ID tidak sinkron antar file: {len(not_sync)}")
    if not not_sync.empty:
        # Cetak ringkas 20 baris pertama agar tidak kepanjangan
        print("\nContoh ID tidak sinkron (maks 20):")
        print(not_sync.head(20))

    # Hanya bandingkan ID yang ada di KETIGANYA
    inner_df = all_df.dropna(subset=["FILE1", "FILE2", "FILE3"])
    print(f"\n- ID yang ada di ketiga file (siap dibandingkan): {len(inner_df)}")

    # Baris yang berbeda nilainya (anggap sama jika ketiganya identik)
    diff_df = inner_df[inner_df.nunique(axis=1) > 1]

    print("\n=== HASIL PERBANDINGAN 'style' (ID hadir di 3 file) ===")
    if diff_df.empty:
        print("Semua nilai 'style' identik untuk setiap 'id' yang hadir di ketiga file.")
    else:
        print(f"Jumlah ID dengan perbedaan: {len(diff_df)}\n")
        pd.set_option("display.max_rows", None)
        pd.set_option("display.max_colwidth", 200)
        print(diff_df)

if __name__ == "__main__":
    main()


=== RINGKASAN ===
- Total baris FILE1: 444
- Total baris FILE2: 444
- Total baris FILE3: 444
- ID tidak sinkron antar file: 0

- ID yang ada di ketiga file (siap dibandingkan): 444

=== HASIL PERBANDINGAN 'style' (ID hadir di 3 file) ===
Jumlah ID dengan perbedaan: 31

                FILE1        FILE2        FILE3
id                                             
Test_003     javanese     javanese     balinese
Test_006     balinese     balinese        dayak
Test_021     balinese     balinese        dayak
Test_040     balinese     balinese        dayak
Test_054     balinese     balinese     javanese
Test_060        batak        batak     javanese
Test_070     balinese     balinese        dayak
Test_075     balinese     balinese     javanese
Test_077  minangkabau  minangkabau     javanese
Test_114  minangkabau  minangkabau        dayak
Test_135     balinese     balinese     javanese
Test_181     balinese     balinese     javanese
Test_191     javanese     javanese  minangkabau
Test_233  